In [ ]:
import numpy as np
import itertools
import math
import scipy
from scipy import spatial
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
from matplotlib import animation
from matplotlib import transforms
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xarray as xr
import dask
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import netCDF4

In [ ]:
def plot_generator_paper(sample, X, Z):
    
    fz = 15*1.25
    lw = 4
    siz = 100
    XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
    XTEXT = 0.25 # Text placement
    YTEXT = 0.3 # Text placement
    
    plt.rc('text', usetex=False)
    matplotlib.rcParams['mathtext.fontset'] = 'stix'
    matplotlib.rcParams['font.family'] = 'STIXGeneral'
    #mpl.rcParams["font.serif"] = "STIX"
    plt.rc('font', family='serif', size=fz)
    matplotlib.rcParams['lines.linewidth'] = lw
    
    
    cmap="RdBu_r"
    fig, ax = plt.subplots(1,1, figsize=(15,6))
    cs0 = ax.pcolor(X, Z, sample, cmap=cmap, vmin=-1.0, vmax = 1.0)
    ax.set_title("Anomalous Vertical Velocity Field Detected By ELBO")
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_xlabel("CRMs", fontsize=fz*1.5)
    ax.xaxis.set_label_coords(0.54,-0.05)
    h = ax.set_ylabel("hPa", fontsize = fz*1.5)
    h.set_rotation(0)
    ax.yaxis.set_label_coords(-0.10,0.44)
    #y_ticks = np.arange(1350, 0, -350)
    #ax.set_yticklabels(y_ticks, fontsize=fz*1.33)
    ax.tick_params(axis='x', labelsize=fz*1.33)
    ax.tick_params(axis='y', labelsize=fz*1.33)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(cs0, cax=cax)
    cbar.set_label(label=r'$\left(\mathrm{m\ s^{-1}}\right)$', rotation="horizontal", fontsize=fz*1.5, labelpad=30, y = 0.65)
    plt.show()
    #plt.savefig("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/CI_Figure_Data/Anomaly.pdf")
    
#plot_generator(test[0,:,:])

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-20-00000.nc'
extra_variables = xr.open_dataset(path_to_file)
lats = np.squeeze(extra_variables.LAT_20s_to_20n.values)
lons = np.squeeze(extra_variables.LON_0e_to_360e.values)

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.20*'
extra_variables = xr.open_mfdataset(path_to_file)
amazon = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[:,:,:,:,10,-29])
atlantic = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[:,:,:,:,10,121])
print(amazon.shape)

In [ ]:
others = netCDF4.Dataset("/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-00000.nc")
levs = np.array(others.variables['lev'])
new = np.flip(levs)
crms = np.arange(1,129,1)
Xs, Zs = np.meshgrid(crms, new)

In [ ]:
Max_Scalar = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Space_Time_Max_Scalar.npy")
Min_Scalar = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Space_Time_Min_Scalar.npy")

In [ ]:
day_images = amazon[16:112,:,:]
week_images = amazon[16:112*6+16,:,:]
synoptic_imagess = amazon[16:112*13,:,:]

atlantic_day_images = atlantic[5:101,:,:]
atlantic_week_images = atlantic[5:96*7+5,:,:]
atlantic_synoptic_imagess = atlantic[5:96*14+5,:,:]

In [ ]:
Test_Day = np.interp(day_images, (Min_Scalar, Max_Scalar), (0, 1))
Test_Week = np.interp(week_images, (Min_Scalar, Max_Scalar), (0, 1))
Test_Synoptic = np.interp(synoptic_imagess, (Min_Scalar, Max_Scalar), (0, 1))

atlantic_Test_Day = np.interp(atlantic_day_images, (Min_Scalar, Max_Scalar), (0, 1))
atlantic_Test_Week = np.interp(atlantic_week_images, (Min_Scalar, Max_Scalar), (0, 1))
atlantic_Test_Synoptic = np.interp(atlantic_synoptic_imagess, (Min_Scalar, Max_Scalar), (0, 1))

In [ ]:
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_test_day.npy",Test_Day)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_test_week.npy",Test_Week)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_test_synoptic.npy",Test_Synoptic)

In [ ]:
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_atlantic_test_day.npy",atlantic_Test_Day)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_atlantic_test_week.npy",atlantic_Test_Week)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_atlantic_test_synoptic.npy",atlantic_Test_Synoptic)

In [ ]:
All_amazon = np.interp(amazon[16:,:,:], (Min_Scalar, Max_Scalar), (0, 1))
All_Atlantic = np.interp(atlantic[5:,:,:], (Min_Scalar, Max_Scalar), (0, 1))

In [ ]:
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_test_amazon_all.npy",All_amazon)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_test_atlantic_all.npy",All_Atlantic)